# Watermarking Example

In this notebook, we will show an example of the watermarking process using the synthid library. This is simply used as a demonstration rather than an analysis.

We will use the following models:
- GPT2
- Gemma 2B
- OLMo 1B
- Llama 3.2 1B

For each model, we will generate text with and without watermarking, and compare the results.

In [1]:
# Imports
import gc

from src.synthid_text import logits_processing
from src.synthid_text import synthid_mixin

import torch
import transformers

from hf_olmo import OLMoForCausalLM, OLMoTokenizerFast

import warnings
warnings.filterwarnings("ignore")

In [2]:
# Hugging Face login
#import huggingface_hub
#huggingface_hub.notebook_login()

In [3]:
# Device
DEVICE = (
    torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")
)
DEVICE

device(type='cpu')

In [4]:
# Config
CONFIG = synthid_mixin.DEFAULT_WATERMARKING_CONFIG
CONFIG

immutabledict({'ngram_len': 5, 'keys': [654, 400, 836, 123, 340, 443, 597, 160, 57, 29, 590, 639, 13, 715, 468, 990, 966, 226, 324, 585, 118, 504, 421, 521, 129, 669, 732, 225, 90, 960], 'sampling_table_size': 65536, 'sampling_table_seed': 0, 'context_history_size': 1024, 'device': device(type='cpu')})

In [15]:
def load_model(
        model_name: str,
        expected_device: torch.device,
        enable_watermarking: bool
) -> transformers.PreTrainedModel:
    if model_name == "gpt2":
        model_cls = (
            synthid_mixin.SynthIDGPT2LMHeadModel
            if enable_watermarking
            else transformers.GPT2LMHeadModel
        )
        model = model_cls.from_pretrained(model_name, device_map="auto")
    elif model_name == "google/gemma-2b-it":
        model_cls = (
            synthid_mixin.SynthIDGemmaForCausalLM
            if enable_watermarking
            else transformers.GemmaForCausalLM
        )
        model = model_cls.from_pretrained(
            model_name,
            device_map="auto",
            torch_dtype=torch.bfloat16,
        )
    elif model_name == "allenai/OLMo-1B":
        model_cls = (
            synthid_mixin.SynthIDOlmoForCausalLM
            if enable_watermarking
            else OLMoForCausalLM
        )
        model = model_cls.from_pretrained(
            model_name,
            revision="step20000-tokens84B",
            device_map='auto',
            torch_dtype=torch.bfloat16
        )
    elif model_name == "meta-llama/Llama-3.2-1B":
        model_cls = (
            synthid_mixin.SynthIDLlamaForCausalLM
            if enable_watermarking
            else transformers.LlamaForCausalLM
        )
        model = model_cls.from_pretrained(
            model_name,
            device_map="auto",
            torch_dtype=torch.bfloat16,
        )
    else:
        model_cls = (
            synthid_mixin.SynthIDAutoModelForCausalLM
            if enable_watermarking
            else transformers.AutoModelForCausalLM
        )
        model = model_cls.from_pretrained(
            model_name,
            device_map="auto",
            torch_dtype=torch.bfloat16,
        )
        
    if str(model.device) != str(expected_device):
        raise ValueError('Model device is not as expected.')
    
    return model

In [16]:
def generate_text(
        model_name: str,
        input_text: str,
        device: torch.device,
        enable_watermarking: bool,
):
    gc.collect()
    torch.cuda.empty_cache()
    
    tokenizer = transformers.AutoTokenizer.from_pretrained(model_name)
    tokenizer.pad_token = tokenizer.eos_token
    tokenizer.padding_side = "left"

    #logits_processor = logits_processing.SynthIDLogitsProcessor(
    #    **CONFIG, top_k=40, temperature=0.7
    #)
    
    inputs = tokenizer(
        input_text,
        return_tensors="pt",
        padding=True,
        return_token_type_ids=False
    ).to(device)
    
    model = load_model(
        model_name, expected_device=device, enable_watermarking=enable_watermarking
    )
    
    torch.manual_seed(0)
    outputs = model.generate(
        **inputs,
        do_sample=True,
        temperature=0.7,
        max_length=1024,
        top_k=40,
        pad_token_id=tokenizer.eos_token_id,
    )
    
    for output in outputs:
        print(tokenizer.decode(output, skip_special_tokens=True))
    
    del inputs, outputs, model
    gc.collect()
    torch.cuda.empty_cache()

In [17]:
# Input text
input_text = ['I enjoy walking with my cute dog']

### GPT2

In [8]:
generate_text(
    model_name='gpt2',
    input_text=input_text,
    device=DEVICE,
    enable_watermarking=False)

I enjoy walking with my cute dog," she said. "He has a lot of aggression and I think he is getting a little bit more aggressive. He is an important part of my life and I appreciate that.

"I have to get out of this place because I love to walk with him," she added.

The dogs were placed in a temporary enclosure after the animal went missing.

A spokesperson for the Queensland Department of Rabbitohs and Wildlife said the animals were being taken to a vet.

"Our animal welfare officers are on scene and our team is investigating," the spokesperson said.

"We have received a report of a missing animal. We are looking at an animal welfare officer's report and will release any information as we can."

Topics: animal-attacks, animals, rudd-7000, qld

First posted


In [9]:
generate_text(
    model_name='gpt2',
    input_text=input_text,
    device=DEVICE,
    enable_watermarking=True)

I enjoy walking with my cute dog. I love playing with his toy, and I love having him come over to my house to play with me. I like to run and play with him. I love the way he looks at me, and I feel like I have the best time with him because I know he has a great personality. I love how he looks like when he is around me. He is a great boy, and I feel like I am trying to be him. I want to make sure I am a good parent too, and make sure I am happy for him. It's very important to me that I treat him like a human being.

What would you say to your 10-year-old daughter, Alyssa?

Alyssa, I think I'm a little too young. I've been through a lot, and it's been tough for me since I started dating her. I think I am a little too young to be with her. I think that she's not getting what I want. I think that she's not getting what I want. I don't think I can be her mom. I think she's not getting what I want.

What advice would you give your daughter about growing up?

I can't give my advice to her.

### Gemma 2B

In [10]:
generate_text(
    model_name='google/gemma-2b-it',
    input_text=input_text,
    device=DEVICE,
    enable_watermarking=False
)

`config.hidden_act` is ignored, you should use `config.hidden_activation` instead.
Gemma's activation function will be set to `gelu_pytorch_tanh`. Please, use
`config.hidden_activation` if you want to override this behaviour.
See https://github.com/huggingface/transformers/pull/29402 for more details.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Some parameters are on the meta device because they were offloaded to the disk and cpu.


I enjoy walking with my cute dog, but I need to be mindful of the distance we can walk while still keeping him safe. I'm not sure how far is safe for him.

Here's a bit more information about my dog:

* **Weight:** 15 pounds
* **Age:** 4 years old
* **Breed:** Golden Retriever
* **Temperament:** Friendly and well-behaved

I'd appreciate any advice you have on the safe walking distance for my dog.


In [11]:
generate_text(
    model_name='google/gemma-2b-it',
    input_text=input_text,
    device=DEVICE,
    enable_watermarking=True
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

I enjoy walking with my cute dog, Bailey. We go on walks together every day, and I love the companionship and the fresh air we breathe while we explore the neighborhood.

However, I recently started noticing some changes in Bailey's behavior. He seems more lethargic, has a lower energy level, and isn't as responsive to my commands. I'm worried about these changes and want to make sure he's healthy and happy.

What are some things I can do to help Bailey?

1. Ensure he has enough exercise and playtime.
2. Make sure he's getting a balanced diet.
3. Make sure he's getting enough mental stimulation.
4. Take him to the vet for a check-up.
5. Consider adding supplements to his diet.


### OLMo 1B

In [12]:
generate_text(
    model_name='allenai/OLMo-1B',
    input_text=input_text,
    device=DEVICE,
    enable_watermarking=False
)

The model weights are not tied. Please use the `tie_weights` method before using the `infer_auto_device` function.


I enjoy walking with my cute dog in the park, but also love the peacefulness and closeness of my family. I have a big heart, because I have been an foster dog for a long time and I need a home where I can fully relax and not be stressed. I'm looking for someone who is caring, loving, and honest, and I would be very happy to provide it. I can adapt to a variety of situations, but I like to have fun and see things from a different perspective. I'm a very kind person, and I want to be able to bring joy to others. I can't wait to meet you!
The ideal match for this dog will be a family with dogs of similar sizes and temperament. There will be a full medical exam and a physical exam. If you think this is the dog for you, please contact us. We will be happy to answer any questions you might have.
This dog is a bit on the small side, though I have a large heart. I am very gentle and kind, and I love to spend time with my family, especially my dog, and I need a dog that can be my constant compa

In [13]:
generate_text(
    model_name='allenai/OLMo-1B',
    input_text=input_text,
    device=DEVICE,
    enable_watermarking=True
)

The model weights are not tied. Please use the `tie_weights` method before using the `infer_auto_device` function.


I enjoy walking with my cute dog, but sometimes the dog will let me down by not paying attention to her commands.
I’m hoping to find some sort of way to train her to not let me down by not paying attention to her commands.
A lot of my training takes place on the treadmill.
It’s the one place where I can go to get my mind off of the fact that my dog isn’t paying attention to me.
It’s also the only place where I can train my dog to not be scared of me.
I can’t help it, I’m still terrified of her.
I hope someone can help me, I’m desperate to get my dog to stop letting me down by not paying attention to her commands.
My dog doesn’t mind me, but if I’m not paying attention to her, I can’t train her to be afraid of me.
I hope someone can help me with this.
I’ve tried everything I can think of, but to no avail.
I have never been able to train my dog to not let me down by not paying attention to her commands.
I’ve also tried to train her to not let me down by not paying attention to her comman

### Llama 3.2 1B

In [18]:
generate_text(
    model_name='meta-llama/Llama-3.2-1B',
    input_text=input_text,
    device=DEVICE,
    enable_watermarking=False
)

I enjoy walking with my cute dog on the beach and exploring the sea. I have been a keen photographer for many years and love capturing the beauty and tranquillity of the North West coastline. I have a special interest in capturing the special moments of family life and enjoy photographing weddings and portraits of children.


In [19]:
generate_text(
    model_name='meta-llama/Llama-3.2-1B',
    input_text=input_text,
    device=DEVICE,
    enable_watermarking=True
)

I enjoy walking with my cute dog. I have two cats. I love to read. I like to sew. I like to cook. I like to bake. I like to travel. I like to dance. I like to listen to music. I like to work. I like to party. I like to swim. I like to watch TV. I like to shop. I like to hang out. I like to play. I like to read books.
I enjoy walking with my cute dog. I have two cats. I love to read. I like to sew. I like to cook. I like to bake. I like to travel. I like to dance. I like to listen to music. I like to work. I like to party. I like to swim. I like to watch TV. I like to shop. I like to hang out. I like to play. I like to read books.
I like to play with my dog. I like to play with my cat. I like to play with my son. I like to play with my daughter. I like to play with my friends.
I like to play with my dog. I like to play with my cat. I like to play with my son. I like to play with my daughter. I like to play with my friends.
I like to play with my dog. I like to play with my cat. I like t